In [1]:
import pandas as pd
import numpy as np

import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_manipulation import transform_data_datetime

## London Grid

- [Copernicus Website](https://cds.climate.copernicus.eu/requests?tab=all)

```
North: 51.6
West:  -0.2
South: 51.4
East:   0.1
```

In [7]:
## GRIB File

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def grib_to_csv_london(grib_file_path, output_csv_path=None):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278  # Note: GRIB files often use 0-360 longitude
    london_lon_360 = 359.8722  # London longitude in 0-360 format
    
    print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Check longitude format (0-360 vs -180-180)
        lon_min = ds.longitude.min().values
        lon_max = ds.longitude.max().values
        print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Use appropriate London longitude based on the dataset's longitude format
        if lon_max > 180:
            target_lon = london_lon_360
            print(f"Using 0-360 longitude format: {target_lon}")
        else:
            target_lon = london_lon
            print(f"Using -180-180 longitude format: {target_lon}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - london_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_lon).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        print(f"Target coordinates: ({london_lat}, {target_lon})")
        print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        london_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        # Add metadata columns
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = target_lon
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            df = df.sort_values(time_cols[0])
        
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        print("\nFirst few rows:")
        print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_london_data.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

def main():
    # Example usage
    grib_file = "./data/ERA5/ERA5_hourly_data_on_single_levels_V1.grib"  # Replace with your GRIB file path
    output_csv = "./output/london_weather_data_V1.csv"  # Replace with desired output path
    
    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    # Process the file
    try:
        df = grib_to_csv_london(grib_file, output_csv)
        
        # Display summary statistics
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        # Load the CSV back into pandas to verify
        print("\n" + "="*50)
        print("VERIFICATION - Loading CSV back into pandas:")
        print("="*50)
        df_loaded = pd.read_csv(output_csv)
        print(f"Loaded DataFrame shape: {df_loaded.shape}")
        print("DataFrame info:")
        print(df_loaded.info())
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")

if __name__ == "__main__":
    main()

Reading GRIB file: ./data/ERA5/ERA5_hourly_data_on_single_levels_V1.grib


c:\ProgramData\anaconda3\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
C:\Users\pacor\AppData\Local\Temp\ipykernel_22512\606618702.py:30: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to 

Dataset loaded successfully!
Dataset dimensions: {'time': 62421, 'step': 2, 'latitude': 413, 'longitude': 2}
Dataset variables: ['tp']
Dataset coordinates: ['number', 'time', 'step', 'surface', 'latitude', 'longitude', 'valid_time']
Longitude range: -0.2 to 0.05
Using -180-180 longitude format: -0.1278
Target coordinates: (51.5074, -0.1278)
Nearest grid point: (51.6, -0.2)
DataFrame shape: (124842, 12)
DataFrame columns: ['time', 'step', 'number', 'surface', 'latitude', 'longitude', 'valid_time', 'tp', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude']

First few rows:
                 time            step  number  surface  latitude  longitude  \
0 1940-01-01 06:00:00 0 days 05:00:00       0      0.0      51.6       -0.2   
1 1940-01-01 06:00:00 0 days 07:00:00       0      0.0      51.6       -0.2   
2 1940-01-01 18:00:00 0 days 05:00:00       0      0.0      51.6       -0.2   
3 1940-01-01 18:00:00 0 days 07:00:00       0      0.0      51.6       -0.2   
4 

In [3]:
df_data = pd.read_csv("./output/london_weather_data_V1.csv")
df_data.tail()

,time,step,number,surface,latitude,longitude,valid_time,tp,actual_latitude,actual_longitude,target_latitude,target_longitude
124837,2025-06-12 06:00:00,0 days 07:00:00,0,0.0,51.6,-0.2,2025-06-12 13:00:00,0.000118,51.6,-0.2,51.5074,-0.1278
124838,2025-06-12 18:00:00,0 days 05:00:00,0,0.0,51.6,-0.2,2025-06-12 23:00:00,0.000000,51.6,-0.2,51.5074,-0.1278
124839,2025-06-12 18:00:00,0 days 07:00:00,0,0.0,51.6,-0.2,2025-06-13 01:00:00,NaN,51.6,-0.2,51.5074,-0.1278
124840,2025-06-13 06:00:00,0 days 05:00:00,0,0.0,51.6,-0.2,2025-06-13 11:00:00,NaN,51.6,-0.2,51.5074,-0.1278
124841,2025-06-13 06:00:00,0 days 07:00:00,0,0.0,51.6,-0.2,2025-06-13 13:00:00,0.000000,51.6,-0.2,51.5074,-0.1278


In [5]:
df_data['year'] = pd.to_datetime(df_data['time'], errors='coerce').dt.year


In [7]:
(df_data
 .groupby('year', observed=True)['tp']
 .sum()
)

year
1940    0.045477
1941    0.042473
1942    0.043283
1943    0.042122
1944    0.042223
          ...   
2021    0.077198
2022    0.054406
2023    0.064124
2024    0.073855
2025    0.020036
Name: tp, Length: 86, dtype: float64

### NC files

In [6]:
# V4 

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Define region around London (in degrees)
        region_size = 0.5  # degrees (approximately 50km at London's latitude)
        
        # Define bounding box around London
        lat_min = london_lat - region_size
        lat_max = london_lat + region_size
        lon_min = london_lon - region_size
        lon_max = london_lon + region_size
        
        print(f"\nExtracting region around London:")
        print(f"Latitude range: {lat_min:.3f} to {lat_max:.3f}")
        print(f"Longitude range: {lon_min:.3f} to {lon_max:.3f}")
        
        # Find grid indices for the region
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find region
                print("Using 2D coordinate arrays to find region")
                
                # Find points within the bounding box
                within_region = ((lat_data >= lat_min) & (lat_data <= lat_max) & 
                               (lon_data >= lon_min) & (lon_data <= lon_max))
                
                # Find the bounding indices of the region
                y_indices, x_indices = np.where(within_region)
                
                if len(y_indices) == 0:
                    print("No grid points found in the specified region!")
                    return None
                
                y_min_idx = y_indices.min()
                y_max_idx = y_indices.max()
                x_min_idx = x_indices.min()
                x_max_idx = x_indices.max()
                
                print(f"Found {len(y_indices)} grid points in region")
                print(f"Grid index ranges: y={y_min_idx}-{y_max_idx}, x={x_min_idx}-{x_max_idx}")
                
                # Store the slice ranges for extraction
                y_slice = slice(y_min_idx, y_max_idx + 1)
                x_slice = slice(x_min_idx, x_max_idx + 1)
                
                # Get actual coordinate bounds of extracted region
                region_lat_min = lat_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].min().values
                region_lat_max = lat_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].max().values
                region_lon_min = lon_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].min().values
                region_lon_max = lon_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].max().values
                
                print(f"Actual extracted region:")
                print(f"  Latitude: {region_lat_min:.3f} to {region_lat_max:.3f}")
                print(f"  Longitude: {region_lon_min:.3f} to {region_lon_max:.3f}")
                
            else:
                # 1D coordinate arrays
                lat_mask = (lat_data >= lat_min) & (lat_data <= lat_max)
                lon_mask = (lon_data >= lon_min) & (lon_data <= lon_max)
                
                y_indices = np.where(lat_mask)[0]
                x_indices = np.where(lon_mask)[0]
                
                if len(y_indices) == 0 or len(x_indices) == 0:
                    print("No grid points found in the specified region!")
                    return None
                
                y_slice = slice(y_indices.min(), y_indices.max() + 1)
                x_slice = slice(x_indices.min(), x_indices.max() + 1)
                
                print(f"Grid slices: y={y_slice}, x={x_slice}")
        else:
            print("Could not find coordinate values for region extraction")
            return None
        
        # Extract data for London region
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: y_slice, proj_x_coord: x_slice}
        london_region_data = ds.isel(selection_dict)
        
        print(f"\nExtracted region data shape: {dict(london_region_data.sizes)}")
        
        print(f"\nExtracted data variables: {list(london_region_data.data_vars)}")
        
        # Convert to pandas DataFrame - simplified approach
        print(f"\nExtracting data for London...")
        
        # Create a simple DataFrame with the main data
        data_dict = {}
        
        # Extract main variables (skip coordinate and bounds variables)
        main_vars = [var for var in london_region_data.data_vars 
                    if not any(skip in var for skip in ['_bnds', 'transverse_mercator', 'bounds'])]
        
        print(f"Main data variables to extract: {main_vars}")
        
        for var_name in main_vars:
            var_data = london_region_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Extract the actual values
            if var_data.size == 1:
                # Single value
                data_dict[var_name] = [float(var_data.values)]
            elif time_coord and time_coord in var_data.dims:
                # Time series data
                values = var_data.values
                if values.ndim > 1:
                    values = values.flatten()
                data_dict[var_name] = values.tolist()
            else:
                # Other cases
                values = var_data.values
                if hasattr(values, 'flatten'):
                    values = values.flatten()
                data_dict[var_name] = [float(values)] if np.isscalar(values) else values.tolist()
            
            # Add metadata
            if hasattr(var_data, 'attrs'):
                units = var_data.attrs.get('units', 'unknown')
                long_name = var_data.attrs.get('long_name', var_name)
                data_dict[f'{var_name}_units'] = [units] * len(data_dict[var_name])
                data_dict[f'{var_name}_long_name'] = [long_name] * len(data_dict[var_name])
        
        # Add time information if available
        if time_coord and time_coord in london_region_data.coords:
            time_values = london_region_data.coords[time_coord].values
            if hasattr(time_values, 'flatten'):
                time_values = time_values.flatten()
            data_dict[time_coord] = time_values.tolist() if hasattr(time_values, 'tolist') else [time_values]
        
        # Create DataFrame
        # First, make sure all lists have the same length
        max_length = max(len(v) if isinstance(v, list) else 1 for v in data_dict.values())
        
        for key, value in data_dict.items():
            if isinstance(value, list) and len(value) == 1 and max_length > 1:
                data_dict[key] = value * max_length
        
        df = pd.DataFrame(data_dict)
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_region_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_region_rainfall_1961_1990_V4.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total grid points extracted: {len(df)}")
            
            # Show spatial extent
            if 'latitude' in df.columns and 'longitude' in df.columns:
                print(f"Latitude range: {df['latitude'].min():.3f} to {df['latitude'].max():.3f}")
                print(f"Longitude range: {df['longitude'].min():.3f} to {df['longitude'].max():.3f}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nRegional precipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
                print(f"Grid points with data: {df[precip_cols[0]].notna().sum()}/{len(df)}")
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

Reading NetCDF file: ./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc
Dataset loaded successfully!
Dataset dimensions: {'time': 1, 'projection_y_coordinate': 1450, 'projection_x_coordinate': 900, 'bnds': 2}
Dataset variables: ['rainfall', 'transverse_mercator', 'time_bnds', 'projection_y_coordinate_bnds', 'projection_x_coordinate_bnds']
Dataset coordinates: ['time', 'projection_y_coordinate', 'projection_x_coordinate', 'latitude', 'longitude']

Dataset overview:
<xarray.Dataset> Size: 31MB
Dimensions:                       (time: 1, projection_y_coordinate: 1450,
                                   projection_x_coordinate: 900, bnds: 2)
Coordinates:
  * time                          (time) datetime64[ns] 8B 1961-07-01
  * projection_y_coordinate       (projection_y_coordinate) float64 12kB -1.9...
  * projection_x_coordinate       (projection_x_coordinate) float64 7kB -1.99...
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 10MB

C:\Users\pacor\AppData\Local\Temp\ipykernel_8912\4207545074.py:31: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dataset dimensions: {dict(ds.dims)}")


Sorted by time column: time

Final DataFrame shape: (8136, 7)
DataFrame columns: ['rainfall', 'rainfall_units', 'rainfall_long_name', 'time', 'target_latitude', 'target_longitude', 'coordinate_system']

Data types:
  rainfall: float64
  rainfall_units: object
  rainfall_long_name: object
  time: int64
  target_latitude: float64
  target_longitude: float64
  coordinate_system: object

First few rows:
        rainfall rainfall_units rainfall_long_name                time  \
0     813.187760             mm     Total rainfall -268358400000000000   
5431  712.100289             mm     Total rainfall -268358400000000000   
5430  699.608436             mm     Total rainfall -268358400000000000   
5429  682.856004             mm     Total rainfall -268358400000000000   
5428  675.770487             mm     Total rainfall -268358400000000000   

      target_latitude  target_longitude    coordinate_system  
0             51.5074           -0.1278  latitude, longitude  
5431          51.5074     

In [4]:
# V3

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find nearest point in 2D space
                print("Using 2D coordinate arrays to find nearest point")
                
                # Calculate distance from London to all grid points
                lat_diff = lat_data - london_lat
                lon_diff = lon_data - london_lon
                distances = np.sqrt(lat_diff**2 + lon_diff**2)
                
                # Find indices of minimum distance
                min_idx = np.unravel_index(distances.argmin(), distances.shape)
                y_idx, x_idx = min_idx
                
                actual_lat = lat_data[y_idx, x_idx].values
                actual_lon = lon_data[y_idx, x_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
                print(f"Grid indices: y={y_idx}, x={x_idx}")
                
                # Use the projection coordinate indices for selection
                lat_idx = y_idx  # This corresponds to projection_y_coordinate index
                lon_idx = x_idx  # This corresponds to projection_x_coordinate index
                
            else:
                # 1D coordinate arrays
                lat_idx = np.abs(lat_data - london_lat).argmin()
                lon_idx = np.abs(lon_data - london_lon).argmin()
                
                actual_lat = lat_data[lat_idx].values
                actual_lon = lon_data[lon_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: lat_idx, proj_x_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame - simplified approach
        print(f"\nExtracting data for London...")
        
        # Create a simple DataFrame with the main data
        data_dict = {}
        
        # Extract main variables (skip coordinate and bounds variables)
        main_vars = [var for var in london_data.data_vars 
                    if not any(skip in var for skip in ['_bnds', 'transverse_mercator', 'bounds'])]
        
        print(f"Main data variables to extract: {main_vars}")
        
        for var_name in main_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Extract the actual values
            if var_data.size == 1:
                # Single value
                data_dict[var_name] = [float(var_data.values)]
            elif time_coord and time_coord in var_data.dims:
                # Time series data
                values = var_data.values
                if values.ndim > 1:
                    values = values.flatten()
                data_dict[var_name] = values.tolist()
            else:
                # Other cases
                values = var_data.values
                if hasattr(values, 'flatten'):
                    values = values.flatten()
                data_dict[var_name] = [float(values)] if np.isscalar(values) else values.tolist()
            
            # Add metadata
            if hasattr(var_data, 'attrs'):
                units = var_data.attrs.get('units', 'unknown')
                long_name = var_data.attrs.get('long_name', var_name)
                data_dict[f'{var_name}_units'] = [units] * len(data_dict[var_name])
                data_dict[f'{var_name}_long_name'] = [long_name] * len(data_dict[var_name])
        
        # Add time information if available
        if time_coord and time_coord in london_data.coords:
            time_values = london_data.coords[time_coord].values
            if hasattr(time_values, 'flatten'):
                time_values = time_values.flatten()
            data_dict[time_coord] = time_values.tolist() if hasattr(time_values, 'tolist') else [time_values]
        
        # Create DataFrame
        # First, make sure all lists have the same length
        max_length = max(len(v) if isinstance(v, list) else 1 for v in data_dict.values())
        
        for key, value in data_dict.items():
            if isinstance(value, list) and len(value) == 1 and max_length > 1:
                data_dict[key] = value * max_length
        
        df = pd.DataFrame(data_dict)
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_rainfall_1961_1990_V3.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

Reading NetCDF file: ./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc
Dataset loaded successfully!
Dataset dimensions: {'time': 1, 'projection_y_coordinate': 1450, 'projection_x_coordinate': 900, 'bnds': 2}
Dataset variables: ['rainfall', 'transverse_mercator', 'time_bnds', 'projection_y_coordinate_bnds', 'projection_x_coordinate_bnds']
Dataset coordinates: ['time', 'projection_y_coordinate', 'projection_x_coordinate', 'latitude', 'longitude']

Dataset overview:
<xarray.Dataset> Size: 31MB
Dimensions:                       (time: 1, projection_y_coordinate: 1450,
                                   projection_x_coordinate: 900, bnds: 2)
Coordinates:
  * time                          (time) datetime64[ns] 8B 1961-07-01
  * projection_y_coordinate       (projection_y_coordinate) float64 12kB -1.9...
  * projection_x_coordinate       (projection_x_coordinate) float64 7kB -1.99...
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 10MB

C:\Users\pacor\AppData\Local\Temp\ipykernel_8912\2614367773.py:31: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dataset dimensions: {dict(ds.dims)}")
C:\Users\pacor\AppData\Local\Temp\ipykernel_8912\2614367773.py:170: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data_dict[var_name] = [float(var_data.values)]


In [ ]:
# Not working for datatype mismatch issues

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find nearest point in 2D space
                print("Using 2D coordinate arrays to find nearest point")
                
                # Calculate distance from London to all grid points
                lat_diff = lat_data - london_lat
                lon_diff = lon_data - london_lon
                distances = np.sqrt(lat_diff**2 + lon_diff**2)
                
                # Find indices of minimum distance
                min_idx = np.unravel_index(distances.argmin(), distances.shape)
                y_idx, x_idx = min_idx
                
                actual_lat = lat_data[y_idx, x_idx].values
                actual_lon = lon_data[y_idx, x_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
                print(f"Grid indices: y={y_idx}, x={x_idx}")
                
                # Use the projection coordinate indices for selection
                lat_idx = y_idx  # This corresponds to projection_y_coordinate index
                lon_idx = x_idx  # This corresponds to projection_x_coordinate index
                
            else:
                # 1D coordinate arrays
                lat_idx = np.abs(lat_data - london_lat).argmin()
                lon_idx = np.abs(lon_data - london_lon).argmin()
                
                actual_lat = lat_data[lat_idx].values
                actual_lon = lon_data[lon_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: lat_idx, proj_x_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Convert to DataFrame
            if time_coord and time_coord in var_data.dims:
                # Time series data
                df_var = var_data.to_dataframe().reset_index()
                
                # Rename the variable column to be more descriptive
                if var_name in df_var.columns:
                    # Keep original variable name but add units if available
                    units = var_data.attrs.get('units', '')
                    long_name = var_data.attrs.get('long_name', var_name)
                    
                    col_name = var_name
                    if units:
                        col_name += f"_{units}".replace(" ", "_")
                    
                    df_var = df_var.rename(columns={var_name: col_name})
                    
                    # Add metadata as separate columns
                    df_var[f'{var_name}_long_name'] = long_name
                    df_var[f'{var_name}_units'] = units
                
                df_list.append(df_var)
            else:
                # Single value or no time dimension
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                
                # Add metadata
                if hasattr(var_data, 'attrs'):
                    for attr_name, attr_value in var_data.attrs.items():
                        df_var[f'{var_name}_{attr_name}'] = str(attr_value)
                
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Find common columns for merging
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols and len(common_cols) > 1:  # More than just the variable column
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # Concatenate side by side if same length
                    if len(df) == len(df_var):
                        df = pd.concat([df, df_var], axis=1)
                    else:
                        print(f"Warning: Cannot merge {df_var.columns} - different lengths")
        else:
            df = df_list[0] if df_list else pd.DataFrame()
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_rainfall_1961_1990_V2.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

Reading NetCDF file: ./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc
Dataset loaded successfully!
Dataset dimensions: {'time': 1, 'projection_y_coordinate': 1450, 'projection_x_coordinate': 900, 'bnds': 2}
Dataset variables: ['rainfall', 'transverse_mercator', 'time_bnds', 'projection_y_coordinate_bnds', 'projection_x_coordinate_bnds']
Dataset coordinates: ['time', 'projection_y_coordinate', 'projection_x_coordinate', 'latitude', 'longitude']

Dataset overview:


C:\Users\pacor\AppData\Local\Temp\ipykernel_8912\4294618856.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dataset dimensions: {dict(ds.dims)}")


<xarray.Dataset> Size: 31MB
Dimensions:                       (time: 1, projection_y_coordinate: 1450,
                                   projection_x_coordinate: 900, bnds: 2)
Coordinates:
  * time                          (time) datetime64[ns] 8B 1961-07-01
  * projection_y_coordinate       (projection_y_coordinate) float64 12kB -1.9...
  * projection_x_coordinate       (projection_x_coordinate) float64 7kB -1.99...
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 10MB ...
    longitude                     (projection_y_coordinate, projection_x_coordinate) float64 10MB ...
Dimensions without coordinates: bnds
Data variables:
    rainfall                      (time, projection_y_coordinate, projection_x_coordinate) float64 10MB ...
    transverse_mercator           int32 4B ...
    time_bnds                     (time, bnds) datetime64[ns] 16B ...
    projection_y_coordinate_bnds  (projection_y_coordinate, bnds) float64 23kB ...
    projectio

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

from pathlib import Path

def grib_to_csv_london(grib_file_path, output_csv_path=None):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278  # Note: GRIB files often use 0-360 longitude
    london_lon_360 = 359.8722  # London longitude in 0-360 format
    
    print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Check longitude format (0-360 vs -180-180)
        lon_min = ds.longitude.min().values
        lon_max = ds.longitude.max().values
        print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Use appropriate London longitude based on the dataset's longitude format
        if lon_max > 180:
            target_lon = london_lon_360
            print(f"Using 0-360 longitude format: {target_lon}")
        else:
            target_lon = london_lon
            print(f"Using -180-180 longitude format: {target_lon}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - london_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_lon).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        print(f"Target coordinates: ({london_lat}, {target_lon})")
        print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        london_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        # Add metadata columns
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = target_lon
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            df = df.sort_values(time_cols[0])
        
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        print("\nFirst few rows:")
        print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_london_data.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

def main():
    # Example usage
    grib_file = "./data/ERA5/ERA5_hourly_data_on_single_levels_from_1940_to_present.grib"  # Replace with your GRIB file path
    output_csv = "./output/london_weather_data.csv"  # Replace with desired output path
    
    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    # Process the file
    try:
        df = grib_to_csv_london(grib_file, output_csv)
        
        # Display summary statistics
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        # Load the CSV back into pandas to verify
        print("\n" + "="*50)
        print("VERIFICATION - Loading CSV back into pandas:")
        print("="*50)
        df_loaded = pd.read_csv(output_csv)
        print(f"Loaded DataFrame shape: {df_loaded.shape}")
        print("DataFrame info:")
        print(df_loaded.info())
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")

if __name__ == "__main__":
    main()

Reading GRIB file: ./data/ERA5/ERA5_hourly_data_on_single_levels_from_1940_to_present.grib


c:\ProgramData\anaconda3\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
C:\Users\pacor\AppData\Local\Temp\ipykernel_8912\3625627430.py:31: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to 

Dataset loaded successfully!
Dataset dimensions: {'time': 2193, 'step': 12, 'latitude': 413, 'longitude': 2}
Dataset variables: ['tp']
Dataset coordinates: ['number', 'time', 'step', 'surface', 'latitude', 'longitude', 'valid_time']
Longitude range: -0.2 to 0.05
Using -180-180 longitude format: -0.1278
Target coordinates: (51.5074, -0.1278)
Nearest grid point: (51.6, -0.2)
DataFrame shape: (26316, 12)
DataFrame columns: ['time', 'step', 'number', 'surface', 'latitude', 'longitude', 'valid_time', 'tp', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude']

First few rows:
                  time            step  number  surface  latitude  longitude  \
0  2021-12-31 18:00:00 0 days 01:00:00       0      0.0      51.6       -0.2   
11 2021-12-31 18:00:00 0 days 12:00:00       0      0.0      51.6       -0.2   
10 2021-12-31 18:00:00 0 days 11:00:00       0      0.0      51.6       -0.2   
9  2021-12-31 18:00:00 0 days 10:00:00       0      0.0      51.6       -0.2  

In [7]:
df_data = pd.read_csv("./output/london_weather_data.csv")
df_data.head()

,time,step,number,surface,latitude,longitude,valid_time,tp,actual_latitude,actual_longitude,target_latitude,target_longitude
0,2021-12-31 18:00:00,0 days 01:00:00,0,0.0,51.6,-0.2,2021-12-31 19:00:00,NaN,51.6,-0.2,51.5074,-0.1278
1,2021-12-31 18:00:00,0 days 12:00:00,0,0.0,51.6,-0.2,2022-01-01 06:00:00,2.868473e-07,51.6,-0.2,51.5074,-0.1278
2,2021-12-31 18:00:00,0 days 11:00:00,0,0.0,51.6,-0.2,2022-01-01 05:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278
3,2021-12-31 18:00:00,0 days 10:00:00,0,0.0,51.6,-0.2,2022-01-01 04:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278
4,2021-12-31 18:00:00,0 days 08:00:00,0,0.0,51.6,-0.2,2022-01-01 02:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278


In [36]:
df_data['year'] = df_data['datetime'].dt.year
df_data['year'].unique()

array([2021, 2022, 2023, 2024])

In [37]:
(df_data
 .groupby('year', observed=True)['tp']
 .sum()
)


year
2021    9.536743e-07
2022    6.779676e-01
2023    7.970974e-01
2024    8.464056e-01
Name: tp, dtype: float64

In [39]:
df_data[df_data['year'] == 2021]

,time,step,number,surface,latitude,longitude,valid_time,tp,actual_latitude,actual_longitude,target_latitude,target_longitude,datetime,year
0,2021-12-31 18:00:00,0 days 01:00:00,0,0.0,51.6,-0.2,2021-12-31 19:00:00,NaN,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
1,2021-12-31 18:00:00,0 days 02:00:00,0,0.0,51.6,-0.2,2021-12-31 20:00:00,NaN,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
2,2021-12-31 18:00:00,0 days 03:00:00,0,0.0,51.6,-0.2,2021-12-31 21:00:00,NaN,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
3,2021-12-31 18:00:00,0 days 04:00:00,0,0.0,51.6,-0.2,2021-12-31 22:00:00,NaN,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
4,2021-12-31 18:00:00,0 days 05:00:00,0,0.0,51.6,-0.2,2021-12-31 23:00:00,NaN,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
5,2021-12-31 18:00:00,0 days 06:00:00,0,0.0,51.6,-0.2,2022-01-01 00:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
6,2021-12-31 18:00:00,0 days 07:00:00,0,0.0,51.6,-0.2,2022-01-01 01:00:00,6.668270e-07,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
7,2021-12-31 18:00:00,0 days 08:00:00,0,0.0,51.6,-0.2,2022-01-01 02:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
8,2021-12-31 18:00:00,0 days 09:00:00,0,0.0,51.6,-0.2,2022-01-01 03:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021
9,2021-12-31 18:00:00,0 days 10:00:00,0,0.0,51.6,-0.2,2022-01-01 04:00:00,0.000000e+00,51.6,-0.2,51.5074,-0.1278,2021-12-31 18:00:00,2021


In [40]:
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            lat_idx = np.abs(ds.coords[lat_coord] - london_lat).argmin()
            lon_idx = np.abs(ds.coords[lon_coord] - london_lon).argmin()
            
            actual_lat = ds.coords[lat_coord][lat_idx].values
            actual_lon = ds.coords[lon_coord][lon_idx].values
            
            print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
            print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        selection_dict = {lat_coord: lat_idx, lon_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Convert to DataFrame
            if time_coord and time_coord in var_data.dims:
                # Time series data
                df_var = var_data.to_dataframe().reset_index()
                
                # Rename the variable column to be more descriptive
                if var_name in df_var.columns:
                    # Keep original variable name but add units if available
                    units = var_data.attrs.get('units', '')
                    long_name = var_data.attrs.get('long_name', var_name)
                    
                    col_name = var_name
                    if units:
                        col_name += f"_{units}".replace(" ", "_")
                    
                    df_var = df_var.rename(columns={var_name: col_name})
                    
                    # Add metadata as separate columns
                    df_var[f'{var_name}_long_name'] = long_name
                    df_var[f'{var_name}_units'] = units
                
                df_list.append(df_var)
            else:
                # Single value or no time dimension
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                
                # Add metadata
                if hasattr(var_data, 'attrs'):
                    for attr_name, attr_value in var_data.attrs.items():
                        df_var[f'{var_name}_{attr_name}'] = str(attr_value)
                
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Find common columns for merging
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols and len(common_cols) > 1:  # More than just the variable column
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # Concatenate side by side if same length
                    if len(df) == len(df_var):
                        df = pd.concat([df, df_var], axis=1)
                    else:
                        print(f"Warning: Cannot merge {df_var.columns} - different lengths")
        else:
            df = df_list[0] if df_list else pd.DataFrame()
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "london_rainfall_1961_1990.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

Reading NetCDF file: ./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc
Dataset loaded successfully!
Dataset dimensions: {'time': 1, 'projection_y_coordinate': 1450, 'projection_x_coordinate': 900, 'bnds': 2}
Dataset variables: ['rainfall', 'transverse_mercator', 'time_bnds', 'projection_y_coordinate_bnds', 'projection_x_coordinate_bnds']
Dataset coordinates: ['time', 'projection_y_coordinate', 'projection_x_coordinate', 'latitude', 'longitude']

Dataset overview:


C:\Users\pacor\AppData\Local\Temp\ipykernel_27988\3345813786.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dataset dimensions: {dict(ds.dims)}")


<xarray.Dataset> Size: 31MB
Dimensions:                       (time: 1, projection_y_coordinate: 1450,
                                   projection_x_coordinate: 900, bnds: 2)
Coordinates:
  * time                          (time) datetime64[ns] 8B 1961-07-01
  * projection_y_coordinate       (projection_y_coordinate) float64 12kB -1.9...
  * projection_x_coordinate       (projection_x_coordinate) float64 7kB -1.99...
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 10MB ...
    longitude                     (projection_y_coordinate, projection_x_coordinate) float64 10MB ...
Dimensions without coordinates: bnds
Data variables:
    rainfall                      (time, projection_y_coordinate, projection_x_coordinate) float64 10MB ...
    transverse_mercator           int32 4B ...
    time_bnds                     (time, bnds) datetime64[ns] 16B ...
    projection_y_coordinate_bnds  (projection_y_coordinate, bnds) float64 23kB ...
    projectio

In [ ]:
import pygrib

# Open GRIB file
grbs = pygrib.open('./data/era5_monthly_averaged_data.grib')

In [ ]:
# grib_file = './data/era5_monthly_averaged_data.grib'
# grbs2 = xr.open_dataset(grib_file, engine='cfgrib') #, backend_kwargs=backend_kwargs)

In [ ]:
# import xarray as xr

# grib_file = './data/era5_monthly_averaged_data.grib'

# # Try the first grid resolution
# try:
#     grbs2 = xr.open_dataset(grib_file, 
#                            engine='cfgrib',
#                            backend_kwargs={'filter_by_keys': {'numberOfPoints': 826}})
#     print("Successfully opened with 826 grid points")
#     print(f"Variables: {list(grbs2.data_vars)}")
#     print(f"Shape: {grbs2.dims}")
# except Exception as e:
#     print(f"Failed with 826 points: {e}")

# # Try the second grid resolution
# try:
#     grbs2_alt = xr.open_dataset(grib_file, 
#                                engine='cfgrib',
#                                backend_kwargs={'filter_by_keys': {'numberOfPoints': 207}})
#     print("Successfully opened with 207 grid points")
#     print(f"Variables: {list(grbs2_alt.data_vars)}")
#     print(f"Shape: {grbs2_alt.dims}")
# except Exception as e:
#     print(f"Failed with 207 points: {e}")

In [ ]:
# grib_file = './data/era5_monthly_averaged_data.grib'

# # Define all combinations to try
# filter_combinations = [
#     {'stepType': 'avgad', 'numberOfPoints': 826},
#     {'stepType': 'avgas', 'numberOfPoints': 826},
#     {'stepType': 'avgad', 'numberOfPoints': 207},
#     {'stepType': 'avgas', 'numberOfPoints': 207}
# ]

# datasets = {}

# for i, filters in enumerate(filter_combinations):
#     try:
#         ds = xr.open_dataset(grib_file, 
#                            engine='cfgrib',
#                            backend_kwargs={'filter_by_keys': filters})
        
#         dataset_name = f"{filters['stepType']}_{filters['numberOfPoints']}pts"
#         datasets[dataset_name] = ds
        
#         print(f"\n✅ Successfully opened: {dataset_name}")
#         print(f"   Variables: {list(ds.data_vars)}")
#         print(f"   Dimensions: {ds.dims}")
#         print(f"   Coordinates: {list(ds.coords)}")
        
#     except Exception as e:
#         print(f"❌ Failed {filters}: {str(e)[:100]}...")

# print(f"\n📊 Total datasets loaded: {len(datasets)}")

In [ ]:
# import xarray as xr
# import pandas as pd

# grib_file = './data/era5_monthly_averaged_data.grib'

# def process_all_era5_data(grib_file):
#     """Process all combinations and return organized data"""
    
#     filter_combinations = [
#         {'stepType': 'avgad', 'numberOfPoints': 826},
#         {'stepType': 'avgas', 'numberOfPoints': 826},
#         {'stepType': 'avgad', 'numberOfPoints': 207},
#         {'stepType': 'avgas', 'numberOfPoints': 207}
#     ]
    
#     all_data = {}
    
#     for filters in filter_combinations:
#         try:
#             # Open dataset
#             ds = xr.open_dataset(grib_file, 
#                                engine='cfgrib',
#                                backend_kwargs={'filter_by_keys': filters})
            
#             # Create descriptive name
#             step_type = filters['stepType']
#             grid_size = filters['numberOfPoints']
#             dataset_name = f"{step_type}_{grid_size}pts"
            
#             # Convert to DataFrame
#             df = ds.to_dataframe().reset_index()
            
#             # Add metadata
#             df['step_type'] = step_type
#             df['grid_points'] = grid_size
#             df['data_type'] = 'accumulated' if step_type == 'avgad' else 'instantaneous'
            
#             all_data[dataset_name] = {
#                 'dataset': ds,
#                 'dataframe': df,
#                 'variables': list(ds.data_vars),
#                 'shape': df.shape
#             }
            
#             print(f"✅ {dataset_name}:")
#             print(f"   Shape: {df.shape}")
#             print(f"   Variables: {list(ds.data_vars)}")
#             print(f"   Sample data types: {df.dtypes.to_dict()}")
            
#         except Exception as e:
#             print(f"❌ Failed {filters}: {e}")
    
#     return all_data

# # Process all data
# all_era5_data = process_all_era5_data(grib_file)

# # Save each dataset to CSV
# for name, data_info in all_era5_data.items():
#     df = data_info['dataframe']
#     filename = f'era5_{name}.csv'
#     df.to_csv(filename, index=False)
#     print(f"💾 Saved {name} to {filename}")

# # Display summary
# print(f"\n📋 Summary:")
# for name, data_info in all_era5_data.items():
#     print(f"{name}: {data_info['shape'][0]} rows, Variables: {data_info['variables']}")


In [ ]:
# # Open the NetCDF file
# ds = xr.open_dataset("./data/london_test_2.nc")

In [ ]:
print(ds)

In [ ]:
rain = ds['tp'].sel(latitude=51.5, longitude=0.0, method='nearest')
rain.plot()
plt.title('Daily Precipitation in London')
plt.show()

In [ ]:
# Select time series at closest grid point to London
rain_series = ds['tp'].sel(latitude=51.5, longitude=0.0, method="nearest")

In [ ]:
# Convert to pandas DataFrame
df = rain_series.to_dataframe().reset_index()
df.head()

In [ ]:
df['RR_mm'] = df['tp'] * 1000
df.head()

In [ ]:
from src.data_manipulation import transform_data_datetime

#london_data = transform_data_datetime(df=df.rename(columns={'valid_time' : 'DATE'}))

In [ ]:
# london_data.head()

In [ ]:
(london_data
 .groupby(['year', 'month'], observed=True)['RR_mm']
 .sum()
 .reset_index()
 )

In [ ]:
london_data['RR_mm'].sum()

In [ ]:
import pygrib

# Open GRIB file
grbs = pygrib.open('./data/era5_hourly_data_on_single_levels.grib')

# # List all messages/variables
# for grb in grbs:
#     print(grb)

In [ ]:
# # Store all data
# all_data = []
    
#     # Process each message
# for grb in grbs:
#         # Filter by variable names if specified
#     # if variable_names and grb.shortName not in variable_names:
#         continue
            
#         # Get data and coordinates
#     data, lats, lons = grb.data()
        
#         # Create coordinate meshgrids
#     lat_grid, lon_grid = np.meshgrid(lats, lons, indexing='ij')
        
#         # Flatten arrays
#     data_flat = data.flatten()
#     lat_flat = lat_grid.flatten()
#     lon_flat = lon_grid.flatten()
        
#         # Get metadata
#     try:
#         valid_date = grb.validDate
#     except:
#         valid_date = None
            
#     try:
#         forecast_time = grb.forecastTime
#     except:
#         forecast_time = 0
            
#         # Create temporary DataFrame for this message
#     temp_df = pd.DataFrame({
#         'latitude': lat_flat,
#         'longitude': lon_flat,
#         'value': data_flat,
#         'variable': grb.shortName,
#         'long_name': getattr(grb, 'name', 'Unknown'),
#         'units': getattr(grb, 'units', 'Unknown'),
#         'level': getattr(grb, 'level', 0),
#         'valid_date': valid_date,
#         'forecast_time': forecast_time
#     })
        
#     all_data.append(temp_df)
    
# grbs.close()

In [ ]:
import pandas as pd

def grib_to_dataframe_pygrib(grib_file, variable_names=None):
    """
    Convert GRIB file to pandas DataFrame using pygrib
    
    Parameters:
    -----------
    grib_file : str
        Path to GRIB file
    variable_names : list, optional
        List of variable short names to extract (e.g., ['tp', 't2m'])
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with meteorological data
    """
    
    # Open GRIB file
    grbs = pygrib.open(grib_file)
    
    # Store all data
    all_data = []
    
    # Process each message
    for grb in grbs:
        # Filter by variable names if specified
        if variable_names and grb.shortName not in variable_names:
            continue
            
        # Get data and coordinates
        data, lats, lons = grb.data()
        
        # Create coordinate meshgrids
        lat_grid, lon_grid = np.meshgrid(lats, lons, indexing='ij')
        
        # Flatten arrays
        data_flat = data.flatten()
        lat_flat = lat_grid.flatten()
        lon_flat = lon_grid.flatten()
        
        print(len(data_flat))
        print(len(lat_flat))
        print(len(lon_flat))

        # Get metadata
        try:
            valid_date = grb.validDate
        except:
            valid_date = None
            
        try:
            forecast_time = grb.forecastTime
        except:
            forecast_time = 0
            
        # Create temporary DataFrame for this message
        print(len(valid_date))
        print(len(forecast_time))

        temp_df = pd.DataFrame({
            'latitude': lat_flat,
            'longitude': lon_flat,
            'value': data_flat,
            'variable': grb.shortName,
            'long_name': getattr(grb, 'name', 'Unknown'),
            'units': getattr(grb, 'units', 'Unknown'),
            'level': getattr(grb, 'level', 0),
            'valid_date': valid_date,
            'forecast_time': forecast_time
        })
        
        all_data.append(temp_df)
    
    grbs.close()
    
    # Combine all data
    if all_data:
        df = pd.concat(all_data, ignore_index=True)
        
        # Remove invalid data points
        df = df.dropna(subset=['value'])
        
        print(f"Dataset shape: {df.shape}")
        print(f"Variables found: {df['variable'].unique()}")
        
        return df
    else:
        print("No data found in GRIB file")
        return pd.DataFrame()

# Example usage:
if __name__ == "__main__":
    # Extract all variables
    df = grib_to_dataframe_pygrib('./data/era5_hourly_data_on_single_levels.grib')
    
    # Extract specific variables only
    df_specific = grib_to_dataframe_pygrib('./data/era5_hourly_data_on_single_levels.grib', 
                                          variable_names=['tp', 't2m', 'r'])
    
    if not df.empty:
        print("\nFirst 5 rows:")
        print(df.head())
        
        # Pivot to have variables as columns
        df_pivot = df.pivot_table(
            index=['latitude', 'longitude', 'valid_date'], 
            columns='variable', 
            values='value'
        ).reset_index()
        
        print("\nPivoted data (variables as columns):")
        print(df_pivot.head())
        
        # Save to CSV
        df.to_csv('weather_data_detailed.csv', index=False)
        df_pivot.to_csv('weather_data_pivot.csv', index=False)

In [ ]:
# Select specific variable
grb = grbs.select(name='Total precipitation')[0]
data, lats, lons = grb.data()

In [ ]:
lats

In [ ]:
lons

In [ ]:
# Read GRIB file
ds = xr.open_dataset('./data/london_data.grib', engine='cfgrib')

In [ ]:
# Try to open only GRIB2 messages
try:
    ds = xr.open_dataset('./data/london_data.grib', 
                        engine='cfgrib',
                        backend_kwargs={'filter_by_keys': {'edition': 2}})
    print("Successfully opened GRIB2 data")
except:
    # If that fails, try GRIB1
    ds = xr.open_dataset('./data/london_data.grib', 
                        engine='cfgrib',
                        backend_kwargs={'filter_by_keys': {'edition': 1}})
    print("Successfully opened GRIB1 data")

print(ds)

In [ ]:
# Explore the dataset
print("Variables:", list(ds.data_vars))
print("Coordinates:", list(ds.coords))

In [ ]:
ds['surface'].values
